In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sklearn.metrics
import lightgbm as lgb
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

from pathlib import Path
import sys
pathlib = str(Path().resolve()) + "/../../"
sys.path.append(pathlib)

data_path = "../../data"
from utils import Timer

! mkdir -p {data_path}/submissions

In [2]:
model = lgb.Booster(model_file = os.path.join(data_path, "1_LearningFE",  "lgbm_trained_HPO1.mdl"))
test_df = pd.read_parquet(os.path.join(data_path, "1_LearningFE",  "test_processed.parquet")).reset_index(drop=True)

selected_features = ['dow']
selected_features += [f"f_{i}" for i in range(0, 80)]
selected_features += [f"f_{i}_CE" for i in [2, 4, 6, 13, 15, 18]+[78, 75, 50, 20, 24]]
selected_features += [f"f_{i}_idx" for i in range(2, 23) if i not in [2, 4, 6, 15]]

target_label = 'is_installed'
excluded_features = ['f_0', 'f_7', 'f_1']

features_train = [i for i in selected_features if i not in excluded_features and i in test_df.columns]

tag = "LFE_dow_ori_CEsubset9_numCE_idx"

with Timer(f"generate submission"):
    pred = model.predict(test_df[features_train])
    submit_df = pd.DataFrame()
    submit_df['RowId'] = test_df['f_0']
    submit_df['is_clicked'] = 0
    submit_df['is_installed'] = pd.Series(pred)
    submit_df.to_csv(os.path.join(data_path, "submissions",  f"Leaderboard_LGBM_{tag}_day_67.csv"), sep = '\t', index = False)
submit_df

generate submission started ...
generate submission took 5.081289602909237 sec


,RowId,is_clicked,is_installed
0,64505,0,0.079024
1,64506,0,0.349310
2,64507,0,0.030132
3,64508,0,0.193603
4,64509,0,0.078676
...,...,...,...
160968,16240,0,0.114474
160969,16241,0,0.004887
160970,16242,0,0.021815
160971,16243,0,0.052142
